[View source on GitHub]: https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/login.ipynb
[Notebook Viewer]: https://nbviewer.jupyter.org/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/login.ipynb
[Run in binder]: https://mybinder.org/v2/gh/wadmp/wadmp.github.io/master?filepath=jupyter_notebooks%2Flogin.ipynb
[Run in Google Colab]: https://colab.research.google.com/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/login.ipynb

| [![GitHub logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/github_logo.png)][View source on GitHub] | [![Jupyter logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/jupyter_logo.png)][Notebook Viewer] | [![binder logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/binder_logo.png)][Run in binder] | [![Colab logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/colab_logo.png)][Run in Google Colab] |
|:---------------------:|:---------------:|:-------------:|:-------------------:|
| [View source on GitHub] | [Notebook Viewer] | [Run in binder] | [Run in Google Colab] |

## Introduction
This notebook provides an example of using the public REST API of WebAccess/DMP.

It prompts the user to enter a Username and Password, and it logs-in to the server.

It also analises 

### Requirements
* If you are running in Jupyter Notebook (>= version 5.3) you don't need to change any code.
* If you are running in Jupyter Lab, you will also need to install these JupyterLab extensions:
  * @jupyter-widgets/jupyterlab-manager
  * jupyter-leaflet
* You need to have an existing user account on the WA/DMP instance.
* In order to see something on the map, you need a device with a cellular connection.
* In order to plot location based on cell tower information, you must have an API key for Google's Geolocation API. (The Geolocation API is free if you make less than 40,000 calls per month).

### Usage
In the "Global Variables" cell below, change BASE_URL to match the particular WA/DMP instance that you are using.

Then run the cells, either one at a time, or all at once.

When prompted, enter the required User Input (USERNAME, PASSWORD, GOOGLE_KEY).

Cell tower location is also marked with a bounding circle. i.e. The device could be anywhere inside this circle.

## Setup

In [ ]:
%%capture

# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pyjwt

import requests
import json
import jwt
import time

## Global variables

In [ ]:
BASE_URL = 'https://gateway.staging.wadmp.com'
BASE_PATH = 'api'

## User input

In [ ]:
USERNAME = input("Enter username:")
PASSWORD = input("Enter password:")

## Login

In [ ]:
url = f"{BASE_URL}/public/auth/connect/token"
credentials = {'username': USERNAME, 'password': PASSWORD, 'client_id': 'python', 'grant_type': 'password'}
print(f"\nSending POST request to {url} with:\n"
        f"    credentials={credentials}")
response = requests.post(url, data=credentials)

print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4, sort_keys=True))
except ValueError:
    print(response.text)

if response.status_code == requests.codes['ok']:
    USER_TOKEN = response.json()["access_token"]
else:
    print("Failed to login!")
    sys.exit(1)

## Analyse the JSON Web Token (JWT)

In [ ]:
headers = jwt.get_unverified_header(USER_TOKEN)
print(json.dumps(headers, indent=4, sort_keys=True))

In [ ]:
claims = jwt.decode(USER_TOKEN, verify=False)
print(json.dumps(claims, indent=4, sort_keys=True))

In [ ]:
nbf = claims['nbf']
print(f"Token valid from {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(nbf))} (local time)")
seconds = int(time.time()) - nbf
print(f"i.e. {seconds} seconds ago")

In [ ]:
exp = claims['exp']
print(f"Token expires {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(exp))} (local time)")
seconds = exp - int(time.time())
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print(f"i.e. In {seconds} seconds, or {h:d}h {m:02d}m {s:02d}s")